# Learning With Noisy Labels via Sparse Regularization

In [ ]:
import torch

In [ ]:
BATCH_SIZE = 100
NUM_WORKERS = 0
RANDOM_SEED = 123
NOISE_RATE = 0.4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# log file
from noise.logger import Logger
model_id = '1'
exp_id = '1-1'
model_name = exp_id + '-' + model_id
log = Logger(mode='debug', title=exp_id)
log.logger.info("{}".format(model_name))

## Data

In [ ]:
from noise.data import NoisyISIC2018
from torchvision import transforms
from torch.utils import data

In [ ]:
# transforms

trans_train = transforms.Compose([
    transforms.CenterCrop((450, 450)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

trans_test = transforms.Compose([
    transforms.CenterCrop((450, 450)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# dataset

train_data = NoisyISIC2018(ann_file='../../Robust-Skin-Lesion-Diagnosis/Data/2018/Train_GroundTruth.csv',
                           img_dir='../../Robust-Skin-Lesion-Diagnosis/Data/2018/ISIC2018_Task3_Training_Input',
                           transform=trans_train, noise_type='asymmetric', noise_rate=NOISE_RATE, random_state=RANDOM_SEED)
test_data = NoisyISIC2018(ann_file='../../Robust-Skin-Lesion-Diagnosis/Data/2018/Test_GroundTruth.csv',
                          img_dir='../../Robust-Skin-Lesion-Diagnosis/Data/2018/ISIC2018_Task3_Training_Input',
                          transform=trans_test, noise_type='symmetric', noise_rate=NOISE_RATE, random_state=RANDOM_SEED)
valid_data = NoisyISIC2018(ann_file='../../Robust-Skin-Lesion-Diagnosis/Data/2018/ISIC2018_Task3_Validation_GroundTruth.csv',
                           img_dir='../../Robust-Skin-Lesion-Diagnosis/Data/2018/ISIC2018_Task3_Validation_Input',
                           transform=trans_test, noise_type='symmetric', noise_rate=NOISE_RATE, random_state=RANDOM_SEED)


In [ ]:
# dataloader

train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=NUM_WORKERS)
test_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=NUM_WORKERS)
valid_loader = data.DataLoader(valid_data, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=NUM_WORKERS)

## Network

In [ ]:
import torch.nn as nn
from torchvision import models
from collections import OrderedDict

In [ ]:
model = models.densenet201(pretrained=True)

# freeze layers
for param in model.parameters():
    param.requires_grad = False


classifier = nn.Sequential(OrderedDict([
    ('fc0', nn.Linear(1920, 256)),
    ('norm0', nn.BatchNorm1d(256)),
    ('relu0', nn.ReLU(inplace=True)),
    ('fc1', nn.Linear(256, 7))
]))

model.classifier = classifier

model.to(device)

## Train

In [ ]:
from noise.train import Trainer
from noise.loss import *
import torch.optim as optim

In [ ]:
# FL + SR
criterion = FocalLoss()
tau, p, lamb, rho, freq = 0.5, 0.01, 5, 1.002, 1
criterion = SR(criterion, tau, p, lamb)

In [ ]:
## hyper-params
init_lr = 1e-3
weight_decay = 1e-4
max_epoch = 100
test_period = 1
early_threshold = 40

optimizer = optim.AdamW(model.classifier.parameters(), lr=init_lr, betas=(0.9, 0.999), weight_decay=weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epoch, eta_min=0)

trainer = Trainer(device, log, model_name, optimizer, scheduler, checkpoint_model=None)

In [ ]:
history = trainer.fit(model, test_loader, valid_loader, criterion, rho, freq, max_epoch, test_period, early_threshold)
print(history)

## Evaluation

In [ ]:
from noise.evaluation import evaluation

In [ ]:
evaluation(model, test_loader)